In [10]:
import re
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde

main_path = "/Users/sebastianhejbergingesen/Downloads/Map"
path = f"{main_path}/stochastic_points_DMS.csv"
df = pd.read_csv(path)
df

latitude     longitude
0     6°56'31.8"S  39°26'37.2"E
1     8°32'36.4"S  38°50'00.9"E
2     7°26'17.3"S  39°01'22.2"E
3     7°26'13.4"S  39°14'10.7"E
4     6°59'35.4"S  38°47'58.1"E
...           ...           ...
2838  1°25'53.8"S  36°32'47.8"E
2839  1°18'02.3"S  36°02'22.6"E
2840  1°46'52.2"S  36°30'47.3"E
2841  1°22'53.4"S  37°07'10.5"E
2842  1°00'16.7"S  36°58'40.3"E

[2843 rows x 2 columns]

In [11]:
def dms_to_decimal(dms):
    pattern = r"(\d+)°(\d+)'([\d.]+)\"?([NSEW])"
    match = re.match(pattern, dms)
    if not match:
        return None
    degrees, minutes, seconds, direction = match.groups()
    decimal = float(degrees) + float(minutes) / 60 + float(seconds) / 3600
    if direction in ['S', 'W']:
        decimal = -decimal
    return decimal

df["latitude"] = df["latitude"].apply(dms_to_decimal)
df["longitude"] = df["longitude"].apply(dms_to_decimal)
df

latitude  longitude
0    -6.942167  39.443667
1    -8.543444  38.833583
2    -7.438139  39.022833
3    -7.437056  39.236306
4    -6.993167  38.799472
...        ...        ...
2838 -1.431611  36.546611
2839 -1.300639  36.039611
2840 -1.781167  36.513139
2841 -1.381500  37.119583
2842 -1.004639  36.977861

[2843 rows x 2 columns]

In [12]:
geometry = [Point(lon, lat) for lat, lon in zip(df["latitude"], df["longitude"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
coordinates = gdf.to_crs(epsg=3857)[["geometry"]]
coordinates

geometry
0      POINT (4390848.888 -774696.29)
1     POINT (4322934.722 -954595.909)
2     POINT (4344001.936 -830345.455)
3     POINT (4367765.555 -830223.836)
4     POINT (4319137.491 -780415.824)
...                               ...
2838  POINT (4068350.139 -159382.805)
2839  POINT (4011911.157 -144798.895)
2840  POINT (4064624.028 -198310.511)
2841   POINT (4132133.115 -153802.78)
2842   POINT (4116356.67 -111841.621)

[2843 rows x 1 columns]

In [ ]:
# Tanzania
tanzania_path_detailed = f"{main_path}/GeoJson/geoBoundaries-TZA-ADM1.geojson"
tanzania_path =   f"{main_path}/GeoJson/geoBoundaries-TZA-ADM2.geojson"

Tanzania = gpd.read_file(tanzania_path).to_crs(epsg=3857)
Tanzania_border = gpd.read_file(tanzania_path_detailed).to_crs(epsg=3857)

# Kenya  
kenya_path_detailed = f"{main_path}/GeoJson/geoBoundaries-KEN-ADM1.geojson"
kenya_path =   f"{main_path}/GeoJson/geoBoundaries-KEN-ADM3.geojson"

Kenya = gpd.read_file(kenya_path).to_crs(epsg=3857)
Kenya_border = gpd.read_file(kenya_path_detailed).to_crs(epsg=3857)

tanzania_border = Tanzania.geometry.union_all().boundary
kenya_border = Kenya.geometry.union_all().boundary

minx, miny, maxx, maxy =  min(Tanzania_border.total_bounds[0], Kenya.total_bounds[0]), \
                          min(Tanzania_border.total_bounds[1], Kenya.total_bounds[1]), \
                          max(Tanzania_border.total_bounds[2], Kenya.total_bounds[2]), \
                          max(Tanzania_border.total_bounds[3], Kenya.total_bounds[3])


fig, ax = plt.subplots(figsize=(15, 15))
Tanzania_border.plot(ax=ax, edgecolor="black", facecolor="white", alpha=1, linewidth=0.8)
Kenya_border.plot(ax=ax, edgecolor="black", facecolor="white", alpha=1, linewidth=0.8)
gpd.GeoSeries([tanzania_border, kenya_border]).plot(ax=ax, edgecolor='black', linewidth=2)
Tanzania.plot(ax=ax, edgecolor=None, facecolor=(96/255,177/255,179/255), alpha=0.76)
Kenya.plot(ax=ax, edgecolor=None, facecolor=(164/255,194/255,217/255), alpha=0.76)

x_vals = [point.x for point in coordinates.geometry]
y_vals = [point.y for point in coordinates.geometry]

xy = np.vstack([x_vals, y_vals])
kde = gaussian_kde(xy)(xy)
norm = mcolors.Normalize(vmin=kde.min(), vmax=kde.max())
colors = [(183/255,228/255,231/255), (243/255, 184/255, 78/255), (244/255, 184/255, 78/255)]
custom_plasma = mcolors.LinearSegmentedColormap.from_list("custom_plasma", colors, N=256)
colors = custom_plasma(norm(kde))
alpha_values = 0.3 + 0.7 * norm(kde)
coordinates.plot(ax=ax, color=colors, markersize=10, alpha=alpha_values)
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)
ax.set_axis_off()
plt.tight_layout()
plt.savefig(f"{main_path}/Map_Kenya_and_Tanzania.pdf", transparent=True, bbox_inches='tight')
plt.show()